In [ ]:
import mimetypes
import pathlib
import shutil
import tempfile

from PIL import Image, ImageOps
# from  pillow_heif import register_heif_opener.  # for mac images
# register_heif_opener()



In [24]:
# pathlib.Path(__file__).resolve() # For python file
NBS_DIR = pathlib.Path().resolve()
REPO_DIR = NBS_DIR.parent
DATA_DIR = REPO_DIR / 'data'
INPUT_DIR = DATA_DIR / 'inputs'
READY_DIR = DATA_DIR / 'ready'
OUTPUT_DIR = DATA_DIR / 'outputs'
GENERATED_DIR = DATA_DIR / 'generated'

In [ ]:
def perform_clear_and_optimize_image(image_path, output_path, max_size=(1920,1920)):

    """
    Removes all metadata from the image ( eg. EXIF data)
    Optimizes the image file size while preserving quality and transparency of the image
    :param image_path:
    :param output_path:
    :param max_size: Fixed size
    :return:
    """

    # convert to path object
    input_path = pathlib.Path(image_path)
    output_path = pathlib.Path(output_path)

    # Open and create a clean copy
    original = Image.open(image_path)

    # Determine if image has transparency
    has_transparency = original.mode in ('RGBA', 'LA') or  (original.mode == 'P' and 'transparency' in original.info)

    # Auto-rotate based on EXIF
    original = ImageOps.exif_transpose(original)

    # Resize if larger than the max_size while maintaining all aspects
    if original.size[0]>max_size[0] or original.size[1]>max_size[1]:
        original.thumbnail(max_size,Image.Resampling.LANCZOS)

    # Convert mode based on transparency
    if has_transparency:
        if original.mode != 'RGBA':
            original = original.convert('RGBA')
        best_format = 'PNG'
    else:
        if original.mode in ("RGBA", "P", "LA"):
            original = original.convert('RGB')
        best_format = 'JPEG'

    # save with optimized settings
    save_kwargs = {}
    if best_format == 'JPEG':
        save_kwargs.update({
           'quality': 85,
            'optimize': True,
            'progressive': True,
        })
        output_path = output_path.with_suffix('.jpg')
    elif best_format == 'PNG':
        save_kwargs.update({
            'optimize': True,
            'compress_level': 6,
        })
        output_path = output_path.with_suffix('.png')
    print(f"Saving {output_path}")
    original.save(output_path, format=best_format, **save_kwargs)
    return output_path




    # image = Image.open(image_path)
    # image = ImageOps.exif_transpose(image)
    # image = ImageOps.autocontrast(image)
    # image = ImageOps.equalize(image)
    # image = ImageOps.posterize(image, 4)
    # image = ImageOps.solarize(image, threshold=128)
    # image = ImageOps.invert(image)
    # image = ImageOps.grayscale(image)
    # image = ImageOps.colorize(image, black="black", white="white")

In [ ]:

def perform_is_image(path, require_can_open=True):

    try:
        guess_type, encoding = mimetypes.guess_type(path)
        guess_type = guess_type or ""
    except:
        guess_type = ""
    print(file_path.name, guess_type)
    guessed_image = "image" in guess_type
    if not guessed_image:
        return False
    if guessed_image and require_can_open:
        try:
            img_ = Image.open(path)
            print(img_)
        except:
            return False
    return True

In [ ]:
image_file_paths = []

for file_path in INPUT_DIR.glob('*'):
    print(file_path)
    is_image = perform_is_image(file_path)
    if not is_image:
        continue
    start_op_path = READY_DIR / file_path.name
    final_op_path = perform_clear_and_optimize_image(file_path, start_op_path)
    image_file_paths.append(final_op_path)

image_file_paths

In [26]:
READY_DIR.mkdir(exist_ok=True, parents=True)
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)
GENERATED_DIR.mkdir(exist_ok=True, parents=True)

In [ ]:
zip_outpath = OUTPUT_DIR / 'images-optimized.zip'
zip_outpath.exists()

In [ ]:
with tempfile.TemporaryDirectory() as temp_dir:
    for path in image_file_paths:
        shutil.copy(path, temp_dir)

    shutil.make_archive(zip_outpath.with_suffix(''), 'zip', temp_dir)

# Generate Image


In [16]:
from decouple import Config, RepositoryEnv, config

config_env = Config(RepositoryEnv(REPO_DIR / '.env'))
REPLICATE_API_TOKEN = config_env('REPLICATE_API_TOKEN', default=None, cast=str)
REPLICATE_MODEL = config('REPLICATE_MODEL', default="slackroo/superme-sumanth-v1")
REPLICATE_MODEL_VERSION = config('REPLICATE_MODEL_VERSION', default="535caef9907be746cc55ccde50aa5eee85d8f987d11c8c4735d70a18d1e37f49", cast=str)

REPLICATE_MODEL

'slackroo/superme-sumanth-v1'

In [17]:
from replicate.client import Client

replicate_client = Client(api_token=REPLICATE_API_TOKEN)

In [18]:
model_f_version = f"{REPLICATE_MODEL}:{REPLICATE_MODEL_VERSION}"
prompt = "a phot of TOK adult man dressed up for a professsional photoshoot."
responses= replicate_client.run(
    model_f_version,
    input={
        "model": "dev",
        "prompt": prompt,
        "go_fast": False,
        "lora_scale": 1,
        "megapixels": "1",
        "num_outputs": 4,
        "aspect_ratio": "1:1",
        "output_format": "jpg",
        "guidance_scale": 3,
        "output_quality": 80,
        "prompt_strength": 0.8,
        "extra_lora_scale": 1,
        "num_inference_steps": 28
    }
)
print(responses)

[<replicate.helpers.FileOutput object at 0x107542a50>, <replicate.helpers.FileOutput object at 0x107542a80>, <replicate.helpers.FileOutput object at 0x107542bd0>, <replicate.helpers.FileOutput object at 0x107542b70>]


In [19]:
len(responses)

4

In [22]:
dir(responses[0])

['__aiter__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_client',
 'aclose',
 'aread',
 'close',
 'read',
 'url']

In [27]:
import random
session_id = random.randint(1, 50_000)
for i , output in enumerate(responses):
    fname= f"{i}-{session_id}.jpg"
    outpath = GENERATED_DIR / fname
    with open(outpath, 'wb') as f:
        f.write(output.read())
    print(i,output)

0 https://replicate.delivery/xezq/Tnovv1ANCnaVPhDoftyq3oAdrDfMYMGZdKyFwf9iqbWQiTDpA/out-0.jpg
1 https://replicate.delivery/xezq/ufoHzHDHZWUOL6sYsuo5WcRoe7HrdSkG3oB4KqKrBhCIxphUA/out-1.jpg
2 https://replicate.delivery/xezq/9AB8D06LVT5eGagvMtDPnXrkROG0Y40oFISR5aun3eyIxphUA/out-2.jpg
3 https://replicate.delivery/xezq/gZyHw9emDxVeCUT2iteuo2B7Li2EDZ6XhTTARxQAyiTRiTDpA/out-3.jpg
